# 1. Basico: Carga de Datos

Importamos las librerias

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

In [ ]:
data = pd.read_csv('/content/OnlineRetail.csv', sep=';')
data



*  Al ejecutar tu analisis exploratorio, describe el numero de columnas y filas que encuentras, en cuales encuentras valores nulos o perdidos y cuantos, Para los valores perdidos numericos:

Numero de filas y columnas

In [ ]:
print('El numero de filas es: ', data.shape[0])
print('El numero de columnas es: ', data.shape[1])

Valores nulos o perdidos

In [ ]:
data.isna().sum()



*   **Elimina todos los valores perdidos utilizando .dropna**

In [ ]:
data.dropna(inplace=True)
print('se elimino correctamente')



*   **Calcula la media, maximo, minimo y desviacion estandar de las columnas con valores numericos.**

In [ ]:
data.describe()



*   **Si encuentras un valor que no tenga logica, es decir valores negativos en columnas donde no deberian haber, eliminalos.**

In [ ]:
print('La cantidad minima nunca debe ser menor a cero')
data = data[data.Quantity >= 0]
data.describe()


*   **Calcula cuantos valores unicos encontraste dentro de cada columna despues de limpiar los datos.

In [ ]:
data.nunique()

# 2. Intermedio: RFM

*   **Calcula la Recenty (Reciente), Frecuency (Frecuencia) y Monetary value (Valor monetario)**

In [ ]:
#Aplicando el tipo de dato a la columna para que se pueda trabajar con las fechas
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

Recenty

In [ ]:
df_recency = data.groupby(by='CustomerID', as_index=False)['InvoiceDate'].max() 
df_recency.columns = ['CustomerID', 'LastPurchaseDate']
recent_date = df_recency['LastPurchaseDate'].max()
df_recency['Recency'] = df_recency['LastPurchaseDate'].apply(lambda x: (recent_date - x).days)
df_recency.head() #Mostramos resultados.

Frecuency

In [ ]:
frequency_df = data.drop_duplicates().groupby(by=['CustomerID'], as_index=False)['InvoiceDate'].count() #Agrupamos por CustomerID sin duplicados
frequency_df.columns = ['CustomerID', 'Frequency'] #Nombramos columnas.
frequency_df.head(10)

Monetary Value

In [ ]:
data['Total'] = data['UnitPrice'] * data['Quantity'] # Calculamos totales multiplicando precio unitario * cantidad
monetary_df = data.groupby(by = 'CustomerID', as_index=False)['Total'].sum() #Agrupamos por CustomerID
monetary_df.columns = ['CustomerID', 'Monetary'] #Creamos columnas
monetary_df.head()

*   **Incluye estos valores en una tabla**

In [ ]:
print('Tabla de Valores')
dataframe = df_recency.merge(frequency_df, on='CustomerID')
df_rfm = dataframe.merge(monetary_df, on='CustomerID').drop(
    columns='LastPurchaseDate')
df_rfm.head()

*  **Calcula los puntajes o scores con un rango de 1 a 5.**

In [ ]:
df_rfm['r_quartile'] = pd.qcut(df_rfm['Recency'], 5, ['5','4','3','2','1'])
df_rfm['f_quartile'] = pd.qcut(df_rfm['Frequency'], 5, ['1','2','3','4','5'])
df_rfm['m_quartile'] = pd.qcut(df_rfm['Monetary'], 5, ['1','2','3','4','5'])
display(df_rfm)